In [1]:
import pandas as pd
import numpy as np
import simplejson
from modules import *

In [2]:
file_csv = "melanoma.csv"

In [3]:
data = pd.read_csv(file_csv).to_numpy()
starting_nodes, starting_node_titles, starting_node_ids = get_starting_nodes(data)
final_pvals = get_pvals_and_children_with_depth(data, starting_node_ids)
log_min_pvals = log_arr(final_pvals[:,0].tolist(),includeNone=True)
mx_log = max(log_arr(final_pvals[:,0].tolist()))
interpolate_vals = np.array([round(x / mx_log, 6) if x is not None else None for x in log_min_pvals])[:, np.newaxis]
final_pvals = np.concatenate(
    [   
        final_pvals[:,:-3],
        np.array(log_min_pvals)[:, np.newaxis],
        interpolate_vals,
        final_pvals[:,-3:-1],
        final_pvals[:,-1][:,np.newaxis]
    ], 
    axis=1
)

In [4]:
final = np.concatenate((starting_nodes, final_pvals), axis=1)
for i, n in np.ndenumerate(final[:,0]):
    final[i[0],0] = n.replace(' ', '_')
final_dataset = pd.DataFrame(
    {
        'Title': final[:,0], 'ID': final[:,1], 
        'min_pval': final[:,2],
        'log_min_pval': final[:,3], 
        'interpolate': final[:,4],
        'init_pval': final[:,5], 
        'min_pval_children': final[:,6],
        'descendants': final[:,-1]
    }
)

In [9]:
csv_table_name = 'to_plunker_' + file_csv
final_dataset.to_csv(csv_table_name, index = False)
plunker_inputs = pd.read_csv(csv_table_name).to_numpy()

In [11]:
if True:
    json_form = ""
    json_attrs = ['Title', 'ID', 'min_pval', 'log_min_pval', 'interpolate']
else:
    json_form = "_long"
    json_attrs = ['Title', 'ID', 'min_pval', 'log_min_pval', 'interpolate', 'init_pval',
              'min_pval_children', 'descendants']

json_data = [{x: plunker_inputs[i,j] for (j, x) in enumerate(json_attrs)}
                for i in range(plunker_inputs.shape[0])]
fileName = "static/local_storage/plunker_inputs_" + file_csv.split(".")[0] + json_form + ".json"
with open(fileName, 'w') as outfile:
    simplejson.dump(json_data, outfile, ignore_nan=True) 